Install Spark 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://www.gtlib.gatech.edu/pub/apache/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

spark-2.4.5-bin-hadoop2.7/
spark-2.4.5-bin-hadoop2.7/licenses/
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-jtransforms.html
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-zstd-jni.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-xmlenc.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-vis.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-spire.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sorttable.js.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-slf4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scopt.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-scala.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-sbt-launch-lib.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-respond.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-reflectasm.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pyrolite.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-py4j.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-protobuf.txt
spark-2.4.5-bin-hadoop2.7/licenses/LICENSE-pmml-model

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
!pip install -q pyspark

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 50.0MB/s 


In [0]:
import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7") # SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

Data Frame 


In [0]:
myrdd_df = spark.read.csv("/content/drive/My Drive/Amazon_Responded_Oct05.csv", header = True, inferSchema=True)
# import pandas as pd
# pd.read_csv("/content/drive/My Drive/Amazon_Responded_Oct05.csv", header=True).shape

In [0]:
myrdd_df.show(5)

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

In [0]:
new_rdd = myrdd_df.select( "tweet_created_at", "user_screen_name", "user_id_str", "text_")

In [0]:
new_rdd = new_rdd.withColumn("day", new_rdd.tweet_created_at.substr(5,6))

In [0]:
new_rdd.show(5)

+--------------------+----------------+-----------+--------------------+------+
|    tweet_created_at|user_screen_name|user_id_str|               text_|   day|
+--------------------+----------------+-----------+--------------------+------+
|Tue Nov 01 01:57:...|     SeanEPanjab|  143515471|@AmazonHelp Can y...|Nov 01|
|Tue Nov 01 02:39:...|      AmazonHelp|   85741735|@SeanEPanjab I'm ...|Nov 01|
|Tue Nov 01 17:14:...|     SeanEPanjab|  143515471|@AmazonHelp It wa...|Nov 01|
|Tue Nov 01 17:15:...|     SeanEPanjab|  143515471|@AmazonHelp I am ...|Nov 01|
|Tue Nov 01 17:19:...|      AmazonHelp|   85741735|@SeanEPanjab Plea...|Nov 01|
+--------------------+----------------+-----------+--------------------+------+
only showing top 5 rows



In [0]:
expression = r"\w{3} \d{2}"
df = new_rdd.filter(new_rdd["day"].rlike(expression ))


In [0]:
df.printSchema()

root
 |-- tweet_created_at: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- text_: string (nullable = true)
 |-- day: string (nullable = true)



In [0]:
df.show()

+--------------------+----------------+-----------+--------------------+------+
|    tweet_created_at|user_screen_name|user_id_str|               text_|   day|
+--------------------+----------------+-----------+--------------------+------+
|Tue Nov 01 01:57:...|     SeanEPanjab|  143515471|@AmazonHelp Can y...|Nov 01|
|Tue Nov 01 02:39:...|      AmazonHelp|   85741735|@SeanEPanjab I'm ...|Nov 01|
|Tue Nov 01 17:14:...|     SeanEPanjab|  143515471|@AmazonHelp It wa...|Nov 01|
|Tue Nov 01 17:15:...|     SeanEPanjab|  143515471|@AmazonHelp I am ...|Nov 01|
|Tue Nov 01 17:19:...|      AmazonHelp|   85741735|@SeanEPanjab Plea...|Nov 01|
|Tue Nov 01 17:25:...|      AmazonHelp|   85741735|@SeanEPanjab With...|Nov 01|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|@AmazonHelp It wa...|Nov 01|
|Tue Nov 01 17:55:...|     SeanEPanjab|  143515471|@AmazonHelp if it...|Nov 01|
|Tue Nov 01 18:02:...|      AmazonHelp|   85741735|@SeanEPanjab I'm ...|Nov 01|
|Tue Nov 01 03:51:...|   aakashwangnoo| 

In [0]:
df.createOrReplaceTempView("Amazon")

In [0]:
# just a test 
df1 = spark.sql("SELECT DISTINCT(day) FROM Amazon WHERE day LIKE 'Apr%' ORDER BY day ASC")
df1.show(5)

+------+
|   day|
+------+
|Apr 01|
|Apr 02|
|Apr 03|
|Apr 04|
|Apr 05|
+------+
only showing top 5 rows



In [0]:
df1.printSchema()

root
 |-- day: string (nullable = true)



In [0]:
#number of days on data set
spark.sql("SELECT COUNT(DISTINCT(day)) FROM Amazon WHERE day NOT LIKE 'has%' ").show()

+-------------------+
|count(DISTINCT day)|
+-------------------+
|                279|
+-------------------+



In [0]:
spark.sql("SELECT * FROM Amazon WHERE day LIKE 'Aug 06' ORDER BY day ASC").show(300)

+--------------------+----------------+-----------+--------------------+------+
|    tweet_created_at|user_screen_name|user_id_str|               text_|   day|
+--------------------+----------------+-----------+--------------------+------+
|Sun Aug 06 03:12:...|     Amtul457BMo| 3214680591|@AmazonHelp Sorry...|Aug 06|
|Sun Aug 06 03:17:...|      AmazonHelp|   85741735|@Amtul457BMo That...|Aug 06|
+--------------------+----------------+-----------+--------------------+------+



In [0]:
spark.sql("SELECT DISTINCT(day) FROM Amazon WHERE user_id_str LIKE '85741735' ORDER BY day ASC").show(5)

+------+
|   day|
+------+
|Apr 01|
|Apr 02|
|Apr 03|
|Apr 04|
|Apr 05|
+------+
only showing top 5 rows



In [0]:
spark.sql('''
SELECT DISTINCT(day) FROM Amazon
WHERE day NOT IN (
  SELECT DISTINCT(day) FROM Amazon WHERE user_id_str LIKE "85741735")  
          ''').show()

+------+
|   day|
+------+
|has 14|
+------+



In [0]:
#NUMBER OF USERS
spark.sql("SELECT COUNT(DISTINCT(user_id_str)) FROM Amazon").show()

+---------------------------+
|count(DISTINCT user_id_str)|
+---------------------------+
|                      71522|
+---------------------------+



In [0]:
spark.sql("SELECT DISTINCT(day) FROM Amazon WHERE user_id_str = '54704557' ").show(5)


+------+
|   day|
+------+
|Feb 15|
|Jun 24|
|Jan 18|
|May 08|
|Jun 23|
+------+
only showing top 5 rows



#TASK 1

In [0]:
#USERS THAT POSTED ALL DAYS 
spark.sql("SELECT user_screen_name, user_id_str FROM Amazon GROUP BY user_id_str, user_screen_name HAVING COUNT(DISTINCT(day)) =279").show()

+----------------+-----------+
|user_screen_name|user_id_str|
+----------------+-----------+
|      AmazonHelp|   85741735|
+----------------+-----------+



In [0]:
#USERS THAT POSTED at least 5 days
daily_active_users = spark.sql("SELECT user_screen_name, user_id_str, COUNT(DISTINCT(day)) as days_active FROM Amazon GROUP BY user_id_str, user_screen_name HAVING COUNT(DISTINCT(day)) >= 5 ORDER BY user_screen_name")
daily_active_users.show()

+----------------+------------------+-----------+
|user_screen_name|       user_id_str|days_active|
+----------------+------------------+-----------+
|          0xJAKE|        4146949275|          5|
|         2SixTwo|        3789082213|          7|
| 353839052d72499|        3025211876|          9|
|   5thLineTalent|          24237942|          5|
|          AW0079|          21064452|         13|
|  Abhinav1bansal|        3315608602|          5|
|        AgentDie|        2200871844|          7|
|  Ahmed_Awad1212|          62705229|          5|
|       AjeetKarn|         278977971|          5|
|    AkiraSakura2|        1064415878|          6|
|  Akshay_goplani|        2889927415|         10|
| Alfonsiawarrior|        2333955555|          7|
|         Alokzgh|733116597825495040|          5|
|      AmazonHelp|          85741735|        279|
|          Amez_W|        1341265634|          5|
|        AmpersUK|         214049826|          5|
| AmyUnitedStates|        2953709027|          5|


In [0]:
daily_active_users.printSchema()

root
 |-- user_screen_name: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- days_active: long (nullable = false)



#TASK 2

In [0]:
experiment = spark.read.csv("/content/drive/My Drive/experiment.txt")

In [0]:
experiment= experiment.selectExpr("_c0 as userid")

In [0]:
experiment.show(5)
experiment.dtypes

+----------+
|    userid|
+----------+
| 143515471|
|  85741735|
|  71457972|
|2908108256|
| 106799492|
+----------+
only showing top 5 rows



[('userid', 'string')]

In [0]:
experiment.printSchema()

root
 |-- userid: string (nullable = true)



In [0]:
#condition to join tables
condition= (experiment.userid == daily_active_users.user_id_str)

In [0]:
#full join
df2 = daily_active_users.join(experiment, condition, how="full")

In [0]:
from pyspark.sql import functions as F


In [0]:
df2 = df2.withColumn("Whether_active", F.when(F.col("user_id_str") == F.col("userid"), "yes").otherwise("no"))

In [0]:
#filter null screen names
df2= df2.filter(df2.user_screen_name.isNotNull())
df2.show()

+----------------+------------------+-----------+------------------+--------------+
|user_screen_name|       user_id_str|days_active|            userid|Whether_active|
+----------------+------------------+-----------+------------------+--------------+
|          0xJAKE|        4146949275|          5|              null|            no|
|         2SixTwo|        3789082213|          7|              null|            no|
| 353839052d72499|        3025211876|          9|              null|            no|
|   5thLineTalent|          24237942|          5|              null|            no|
|          AW0079|          21064452|         13|          21064452|           yes|
|  Abhinav1bansal|        3315608602|          5|              null|            no|
|        AgentDie|        2200871844|          7|              null|            no|
|  Ahmed_Awad1212|          62705229|          5|              null|            no|
|       AjeetKarn|         278977971|          5|              null|        

In [0]:
experiment_user = df2.drop("user_id_str", "userid")

In [0]:
experiment_user = experiment_user.selectExpr("user_screen_name as user_id_str", "Whether_active")
experiment_user.show(20)

+---------------+--------------+
|    user_id_str|Whether_active|
+---------------+--------------+
|         0xJAKE|            no|
|        2SixTwo|            no|
|353839052d72499|            no|
|  5thLineTalent|            no|
|         AW0079|           yes|
| Abhinav1bansal|            no|
|       AgentDie|            no|
| Ahmed_Awad1212|            no|
|      AjeetKarn|            no|
|   AkiraSakura2|            no|
| Akshay_goplani|            no|
|Alfonsiawarrior|            no|
|        Alokzgh|            no|
|     AmazonHelp|           yes|
|         Amez_W|           yes|
|       AmpersUK|           yes|
|AmyUnitedStates|            no|
|   AndyMRoberts|           yes|
|          Aosc2|           yes|
|       Arthi305|           yes|
+---------------+--------------+
only showing top 20 rows



In [0]:
experiment_user.printSchema()

root
 |-- user_id_str: string (nullable = true)
 |-- Whether_active: string (nullable = false)



In [0]:
experiment_user.createOrReplaceTempView("experiment_user")

In [0]:
#total count of Whether_active
spark.sql("SELECT count(Whether_active) FROM experiment_user ").show()


+---------------------+
|count(Whether_active)|
+---------------------+
|                  593|
+---------------------+



In [0]:
#count of Whether_active = yes
spark.sql('SELECT count(Whether_active) FROM experiment_user WHERE Whether_active == "yes"  ').show()

+---------------------+
|count(Whether_active)|
+---------------------+
|                  121|
+---------------------+



In [0]:
#calculate percentage of active user 
121/593

0.20404721753794267

#TASK 3


Select the records (all columns) in
Amazon_Responded_Oct05.csv when a user_id_str is included in all the 2 dataframes.


[daily_active_users & experiment_user]. 


Save the result in a dataframe and then export it as Amazon_new.csv

In [0]:
myrdd_df.show()

+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+-------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweeted|                text|
+--------------------+--------------------+----------------+-----------+-------------------+--

In [0]:
experiment_user.show()

+---------------+--------------+
|    user_id_str|Whether_active|
+---------------+--------------+
|         0xJAKE|            no|
|        2SixTwo|            no|
|353839052d72499|            no|
|  5thLineTalent|            no|
|         AW0079|           yes|
| Abhinav1bansal|            no|
|       AgentDie|            no|
| Ahmed_Awad1212|            no|
|      AjeetKarn|            no|
|   AkiraSakura2|            no|
| Akshay_goplani|            no|
|Alfonsiawarrior|            no|
|        Alokzgh|            no|
|     AmazonHelp|           yes|
|         Amez_W|           yes|
|       AmpersUK|           yes|
|AmyUnitedStates|            no|
|   AndyMRoberts|           yes|
|          Aosc2|           yes|
|       Arthi305|           yes|
+---------------+--------------+
only showing top 20 rows



In [0]:
daily_active_users.show()

+----------------+------------------+-----------+
|user_screen_name|       user_id_str|days_active|
+----------------+------------------+-----------+
|          0xJAKE|        4146949275|          5|
|         2SixTwo|        3789082213|          7|
| 353839052d72499|        3025211876|          9|
|   5thLineTalent|          24237942|          5|
|          AW0079|          21064452|         13|
|  Abhinav1bansal|        3315608602|          5|
|        AgentDie|        2200871844|          7|
|  Ahmed_Awad1212|          62705229|          5|
|       AjeetKarn|         278977971|          5|
|    AkiraSakura2|        1064415878|          6|
|  Akshay_goplani|        2889927415|         10|
| Alfonsiawarrior|        2333955555|          7|
|         Alokzgh|733116597825495040|          5|
|      AmazonHelp|          85741735|        279|
|          Amez_W|        1341265634|          5|
|        AmpersUK|         214049826|          5|
| AmyUnitedStates|        2953709027|          5|


In [0]:
#joining the 3 tables 
condition1= (experiment_user.user_id_str == daily_active_users.user_screen_name)
condition2= (daily_active_users.user_screen_name == myrdd_df.user_screen_name)

df6 = daily_active_users.join(experiment_user, condition1, how="inner").join(myrdd_df, condition2, how="right")
df7= df6.filter(df.user_screen_name.isNotNull())
df7.show()


+----------------+-----------+-----------+--------------+--------------+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+------------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+
|user_screen_name|user_id_str|days_active|   user_id_str|Whether_active|              id_str|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|     user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_

In [0]:
import pandas as pd

df8 = df7.toPandas()

In [0]:
df8 = df8.iloc[ :, 5:30]
df8.head()

,user_favourites_count,user_protected,user_listed_count,user_following,user_description,user_location,user_verified,user_followers_count,user_friends_count,user_created_at,tweet_language,text_,favorite_count,favorited,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,retweeted,text
0,2492,False,29,False,Charity comms bod | @thegreenparty candidate f...,"Lewisham, London,",False,629,1190,Tue Jan 17 10:35:54 +0000 2012,en,Mum's @AmazonUK order from weeks ago was lost ...,0,False,None,None,None,0,False,None
1,2492,False,29,False,Charity comms bod | @thegreenparty candidate f...,"Lewisham, London,",False,629,1190,Tue Jan 17 10:35:54 +0000 2012,en,@AmazonHelp it's from ages ago. She received h...,0,False,AmazonHelp,8.130052269827359e+17,85741735.0,0,False,None
2,2492,False,29,False,Charity comms bod | @thegreenparty candidate f...,"Lewisham, London,",False,629,1190,Tue Jan 17 10:35:54 +0000 2012,en,@AmazonHelp and you totally shouldn't be worki...,0,False,AmazonHelp,8.130064191842877e+17,85741735.0,0,False,None
3,2492,False,29,False,Charity comms bod | @thegreenparty candidate f...,"Lewisham, London,",False,629,1190,Tue Jan 17 10:35:54 +0000 2012,en,@AmazonHelp the order doesn't show in that lis...,0,False,AmazonHelp,8.130064191842877e+17,85741735.0,0,False,None
4,2492,False,29,False,Charity comms bod | @thegreenparty candidate f...,"Lewisham, London,",False,629,1190,Tue Jan 17 10:35:54 +0000 2012,en,@AmazonHelp ok I've found it - but it's not cl...,0,False,AmazonHelp,8.130064191842877e+17,85741735.0,0,False,@AmazonHelp ok I've found it - but it's not cl...


In [0]:
df8.to_csv("/content/drive/My Drive/IDS561/Amazon_new.csv", header = True, index=False)